In [39]:
import shutil
import os
import glob
import random
from tqdm import tqdm
from collections import defaultdict

In [40]:
# 랜덤 시드 설정
RANDOM_SEED = 2024
random.seed(RANDOM_SEED)

# AI-Hub의 낙상사고 위험동작 영상-센서 쌍 데이터를 다운로드 (https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71641)
# D:\Falldown\Dataset\Original_Dataset를 다운로드 경로로 지정
# 01.원천데이터 -> SourceData (폴더 이름 변경)
# 02.라벨링데이터 -> LabelingData (폴더 이름 변경)

In [ ]:
# 파일 복사 함수
def copy_files(destination, groups, group_dict, source=None):
    moved_files = 0  # 이동된 파일 수
    skipped_files = 0  # 건너뛴 파일 수
    if not os.path.exists(destination):
        os.makedirs(destination)

    for group in tqdm(groups, desc=f"{source}에서 {destination}로 파일 복사 중"):
        for file_path in group_dict[group]:
            dest_path = os.path.join(destination, os.path.basename(file_path))
            if os.path.exists(dest_path):  # 파일이 이미 존재하는 경우
                skipped_files += 1
                continue  # 복사하지 않고 건너뜀

            shutil.copy(file_path, dest_path)
            moved_files += 1

    # 작업 결과 출력
    print(f"'{destination}'에 파일 복사 완료: {moved_files}개 파일 복사, {skipped_files}개 파일 건너뜀.")
    return moved_files

In [42]:
# Source 경로
source_folders_N = glob.glob('D:\\Falldown\\Dataset\\Original_Dataset\\*\\SourceData\\Image\\N', recursive=True)
source_folders_Y = glob.glob('D:\\Falldown\\Dataset\\Original_Dataset\\*\\SourceData\\Image\\Y', recursive=True)

# Destination 경로
base_dataset_dir = 'D:\\Falldown\\Dataset\\Sorted_Dataset'
train_folder = os.path.join(base_dataset_dir, 'Train', 'Image')
val_folder = os.path.join(base_dataset_dir, 'Val', 'Image')
test_folder = os.path.join(base_dataset_dir, 'Test', 'Image')

In [43]:
# 그룹화
file_groups_N = defaultdict(list)
file_groups_Y = defaultdict(list)

for folder in source_folders_N:
    for file_path in glob.glob(f"{folder}\\**\\*.jpg", recursive=True):
        group_key = os.path.basename(file_path).rsplit('_I', 1)[0]
        file_groups_N[group_key].append(file_path)

for folder in source_folders_Y:
    for file_path in glob.glob(f"{folder}\\**\\*.jpg", recursive=True):
        group_key = os.path.basename(file_path).rsplit('_I', 1)[0]
        file_groups_Y[group_key].append(file_path)

"""
예시)
file_groups = {
    "00050_H_A_BY_C1": [
        r"D:\Falldown\Dataset\Original_Dataset\Training\SourceData\Image\Y\BY\00050_H_A_BY_C1\00050_H_A_BY_C1_I001.jpg",
        r"D:\Falldown\Dataset\Original_Dataset\Training\SourceData\Image\Y\BY\00050_H_A_BY_C1\00050_H_A_BY_C1_I002.jpg"
    ]
}
"""

'\n예시)\nfile_groups = {\n    "00050_H_A_BY_C1": [\n        r"D:\\Falldown\\Dataset\\Original_Dataset\\Training\\SourceData\\Image\\Y\\BY\x0050_H_A_BY_C1\x0050_H_A_BY_C1_I001.jpg",\n        r"D:\\Falldown\\Dataset\\Original_Dataset\\Training\\SourceData\\Image\\Y\\BY\x0050_H_A_BY_C1\x0050_H_A_BY_C1_I002.jpg"\n    ]\n}\n'

In [ ]:
# 데이터 분할 함수
def split_data(file_groups, split_ratios):
    group_keys = list(file_groups.keys())
    random.shuffle(group_keys)

    train_end = int(len(group_keys) * split_ratios[0])
    val_end = train_end + int(len(group_keys) * split_ratios[1])

    train_groups = group_keys[:train_end]
    val_groups = group_keys[train_end:val_end]
    test_groups = group_keys[val_end:]

    return train_groups, val_groups, test_groups

# Train, Validation, Test 비율
SPLIT_RATIOS = (0.8, 0.1, 0.1)

# N 데이터 분할
train_groups_N, val_groups_N, test_groups_N = split_data(file_groups_N, SPLIT_RATIOS)

# Y 데이터에서 사용할 그룹 키를 1/3로 줄이고 새롭게 그룹화
reduced_y_keys = random.sample(list(file_groups_Y.keys()), len(file_groups_Y) // 3) # 전체 키 개수의 1/3만큼 랜덤으로 선택
file_groups_Y = {key: file_groups_Y[key] for key in reduced_y_keys}

# Y 데이터 분할
train_groups_Y, val_groups_Y, test_groups_Y = split_data(file_groups_Y, SPLIT_RATIOS)

In [45]:
# N 데이터 복사
train_files_copied_N = copy_files(destination=train_folder, groups=train_groups_N, group_dict=file_groups_N, source="N 데이터")
val_files_copied_N = copy_files(destination=val_folder, groups=val_groups_N, group_dict=file_groups_N, source="N 데이터")
test_files_copied_N = copy_files(destination=test_folder, groups=test_groups_N, group_dict=file_groups_N, source="N 데이터")

# Y 데이터 복사
train_files_copied_Y = copy_files(destination=train_folder, groups=train_groups_Y, group_dict=file_groups_Y, source="Y 데이터")
val_files_copied_Y = copy_files(destination=val_folder, groups=val_groups_Y, group_dict=file_groups_Y, source="Y 데이터")
test_files_copied_Y = copy_files(destination=test_folder, groups=test_groups_Y, group_dict=file_groups_Y, source="Y 데이터")

# 총 파일 개수 요약 출력
total_train_files = train_files_copied_N + train_files_copied_Y
total_val_files = val_files_copied_N + val_files_copied_Y
total_test_files = test_files_copied_N + test_files_copied_Y

print(f"\n결과 요약:")
print(f"Train Set: {total_train_files} files (N: {train_files_copied_N}, Y: {train_files_copied_Y})")
print(f"Validation Set: {total_val_files} files (N: {val_files_copied_N}, Y: {val_files_copied_Y})")
print(f"Test Set: {total_test_files} files (N: {test_files_copied_N}, Y: {test_files_copied_Y})")

N 데이터에서 D:\Falldown\Dataset\Sorted_Dataset\Train\Image로 파일 복사 중: 100%|██████████| 4089/4089 [05:19<00:00, 12.81it/s]
N 데이터에서 D:\Falldown\Dataset\Sorted_Dataset\Val\Image로 파일 복사 중: 100%|██████████| 511/511 [00:41<00:00, 12.34it/s]
N 데이터에서 D:\Falldown\Dataset\Sorted_Dataset\Test\Image로 파일 복사 중: 100%|██████████| 512/512 [00:42<00:00, 12.03it/s]
Y 데이터에서 D:\Falldown\Dataset\Sorted_Dataset\Train\Image로 파일 복사 중: 100%|██████████| 4076/4076 [06:04<00:00, 11.18it/s]
Y 데이터에서 D:\Falldown\Dataset\Sorted_Dataset\Val\Image로 파일 복사 중: 100%|██████████| 509/509 [00:44<00:00, 11.48it/s]
Y 데이터에서 D:\Falldown\Dataset\Sorted_Dataset\Test\Image로 파일 복사 중: 100%|██████████| 511/511 [00:44<00:00, 11.42it/s]


결과 요약:
Train Set: 81650 files (N: 40890, Y: 40760)
Validation Set: 10200 files (N: 5110, Y: 5090)
Test Set: 10230 files (N: 5120, Y: 5110)


In [ ]:
# Sorted_Dataset으로 옮겨온 Image와 동일한 이름의 JSON 파일을 label 폴더로 전부 복사
def copy_json_files(base_dir):
    folders = ['Train', 'Val', 'Test']
    json_files = glob.glob('D:\\Falldown\\Dataset\\Original_Dataset\\*\\LabelingData\\Image\\**\\*.json', recursive=True)

    if not json_files:
        print("Warning: Source 경로에 JSON file이 없습니다.")
        return

    # json_files를 {파일이름: 경로} 형식의 딕셔너리로 변환
    json_dict = {os.path.splitext(os.path.basename(json_file))[0]: json_file for json_file in json_files}

    skipped_json = 0  # 이미 존재하는 JSON 파일 수
    copied_json = 0  # 새로 복사된 JSON 파일 수

    for folder in folders:
        images_dir = os.path.join(base_dir, folder, 'Image')
        labels_dir = os.path.join(base_dir, folder, 'Label')

        if not os.path.exists(labels_dir):
            os.makedirs(labels_dir)

        # Image 폴더의 모든 이미지 파일에 대해
        image_files = os.listdir(images_dir)
        if not image_files:
            print(f"Warning: {images_dir} 디렉토리에 이미지 파일이 없습니다.")
            continue

        # 이미지 파일의 이름만 추출 (확장자 제외)
        for image_file in tqdm(image_files, desc=f"Processing {folder} JSON files"):
            image_name = os.path.splitext(image_file)[0]
            # 동일한 이름의 JSON 파일이 딕셔너리에 존재하면 labels 폴더로 복사
            if image_name in json_dict:
                json_path = json_dict[image_name]
                destination_path = os.path.join(labels_dir, f"{image_name}.json")

                try:
                    if os.path.exists(destination_path):  # 파일이 이미 존재하면 건너뜀
                        skipped_json += 1
                        continue
                    shutil.copy(json_path, destination_path)
                    copied_json += 1
                except Exception as e:
                    print(f"Error: {image_name}.json를 복사하는 중 문제가 발생했습니다. {e}")
            else:
                print(f"Warning: {image_file}에 해당되는 JSON 파일이 없습니다.")

    # 처리 결과 요약 출력
    print(f"총 JSON 파일 처리 완료: {copied_json}개 파일을 복사하고, {skipped_json}개 파일을 건너뛰었습니다.")

# 경로 지정
base_directory = 'D:\\Falldown\\Dataset\\Sorted_Dataset'  # Image를 옮겨왔던 Sorted_Dataset 경로

copy_json_files(base_directory)

Processing Test JSON files: 100%|██████████| 10230/10230 [00:09<00:00, 1074.01it/s]
